In [1]:
# Standard imports
import pandas as pd
import numpy as np
import os
import scipy.stats as stats

# For inserting random values into nulls if necessary
import random

# To acquire MYSQL Data
import acquire
from env import username, password, host
import wrangle

import warnings
warnings.filterwarnings('ignore')

# For data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# For running modeling
from sklearn.model_selection import train_test_split
import sklearn.metrics as mtc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer

# Exercises
##### Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

### As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

## In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

#### 1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

#### 2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

#### 3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [2]:
acquire.get_zillow_2017().head(10)

,Unnamed: 0,propertylandusetypeid,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,propertylandusedesc
0,0,269.0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,Planned Unit Development
1,1,261.0,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,Single Family Residential
2,2,47.0,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,...,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN,Store/Office (Mixed Use)
3,3,47.0,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,...,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN,Store/Office (Mixed Use)
4,4,31.0,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,...,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN,Commercial/Office/Residential Mixed Used
5,5,31.0,5,10898347,NaN,NaN,NaN,0.0,0.0,4.0,...,NaN,179072.0,287634.0,2016.0,108562.0,3661.28,NaN,NaN,NaN,Commercial/Office/Residential Mixed Used
6,6,260.0,6,10933547,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,404013.0,563029.0,2016.0,159016.0,6773.34,NaN,NaN,NaN,Residential General
7,7,31.0,7,10940747,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,103553.0,698984.0,2016.0,595431.0,7857.84,NaN,NaN,NaN,Commercial/Office/Residential Mixed Used
8,8,269.0,8,10954547,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,Planned Unit Development
9,9,31.0,9,10976347,NaN,NaN,NaN,0.0,0.0,3.0,...,NaN,221771.0,265184.0,2016.0,43413.0,4054.76,NaN,NaN,NaN,Commercial/Office/Residential Mixed Used


In [3]:
# Dropping obvious unnecessary columns right off the bat
zillow17 = acquire.get_zillow_2017().drop(columns={'id', 'Unnamed: 0'})
zillow17.head(3)

,propertylandusetypeid,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,propertylandusedesc
0,269.0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,Planned Unit Development
1,261.0,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,Single Family Residential
2,47.0,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN,Store/Office (Mixed Use)


In [4]:
# Split the data and verify sizes by checking their shapes
train, validate, test = wrangle.split(zillow17)

df shape: (2982285, 59)
Train shape: (1789371, 59)
Validate shape: (596457, 59)
Test shape: (596457, 59)


---

In [ ]:
# Sending the data to a .csv file for future use and quicker pulls
train.to_csv('zillow_train.csv')
validate.to_csv('zillow_validate.csv')
test.to_csv('zillow_test.csv')

In [5]:
# I am specifically looking for SFH in these exercises
train.propertylandusedesc.value_counts()

Single Family Residential                     1292340
Condominium                                    289724
Duplex (2 Units, Any Combination)               68338
Planned Unit Development                        36893
Mobile Home                                     35724
Quadruplex (4 Units, Any Combination)           24532
Triplex (3 Units, Any Combination)              24006
Commercial/Office/Residential Mixed Used         5696
Cluster Home                                     5675
Store/Office (Mixed Use)                         2634
Residential General                              1800
Cooperative                                      1100
Manufactured, Modular, Prefabricated Homes        729
Townhouse                                         156
Residential Common Area                            23
Inferred Single Family Residential                  1
Name: propertylandusedesc, dtype: int64

In [6]:
# Narrow down the data frame and reassign it back into the variable
train = train[(train.propertylandusedesc == 'Single Family Residential') | (train.propertylandusedesc == 'Inferred Single Family Residential')]

In [7]:
train.head(3)

,propertylandusetypeid,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,propertylandusedesc
2236567,261.0,11216329,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,NaN,212207.0,265257.0,2016.0,53050.0,5537.14,NaN,NaN,6.037911e+13,Single Family Residential
1804351,261.0,17147507,NaN,NaN,NaN,3.0,5.0,NaN,NaN,3.0,...,NaN,224000.0,642000.0,2016.0,418000.0,7906.32,NaN,NaN,6.111003e+13,Single Family Residential
1861128,261.0,14142821,NaN,NaN,NaN,3.5,6.0,NaN,NaN,3.5,...,NaN,168445.0,822352.0,2016.0,653907.0,9254.54,NaN,NaN,6.059002e+13,Single Family Residential


In [8]:
sfh_train = train[['bedroomcnt','bathroomcnt','calculatedfinishedsquarefeet','taxvaluedollarcnt','yearbuilt','taxamount','fips']]

In [9]:
sfh_train.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
2236567,3.0,3.0,2988.0,265257.0,2006.0,5537.14,6037.0
1804351,5.0,3.0,2826.0,642000.0,2005.0,7906.32,6111.0
1861128,6.0,3.5,2850.0,822352.0,1966.0,9254.54,6059.0
807,4.0,2.0,1576.0,217168.0,1971.0,2532.56,6111.0
1004951,3.0,3.0,2395.0,219511.0,1939.0,2712.17,6037.0


In [10]:
# Looking for null values
sfh_train.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1292341 entries, 2236567 to 2594527
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    1292337 non-null  float64
 1   bathroomcnt                   1292337 non-null  float64
 2   calculatedfinishedsquarefeet  1287285 non-null  float64
 3   taxvaluedollarcnt             1292055 non-null  float64
 4   yearbuilt                     1286778 non-null  float64
 5   taxamount                     1289667 non-null  float64
 6   fips                          1292341 non-null  float64
dtypes: float64(7)
memory usage: 78.9 MB


In [11]:
# Getting a sum of all the nulls in each column
sfh_train.isnull().sum()

bedroomcnt                         4
bathroomcnt                        4
calculatedfinishedsquarefeet    5056
taxvaluedollarcnt                286
yearbuilt                       5563
taxamount                       2674
fips                               0
dtype: int64

In [12]:
# Using this for loop to show the number of nulls in each column as well as the percentage of that number, 
# compared to the rest of the non-null values
for col in sfh_train:
    print(f'{col} null count: {sfh_train[col].isnull().sum()}')
    print(sfh_train[col].isnull().sum()/len(sfh_train))
    print('------')

bedroomcnt null count: 4
3.095158321217078e-06
------
bathroomcnt null count: 4
3.095158321217078e-06
------
calculatedfinishedsquarefeet null count: 5056
0.0039122801180183865
------
taxvaluedollarcnt null count: 286
0.0002213038199670211
------
yearbuilt null count: 5563
0.004304591435232651
------
taxamount null count: 2674
0.0020691133377336167
------
fips null count: 0
0.0
------


In [13]:
# Utilizing this function to find the info on each column that'll help direct what value to input for nulls if necessary
def v(df):    
    for col in df:
        print(f'Column: {col}')
        print(f'Min: {df[col].min()}')
        print(f'Max: {df[col].max()}')
        print(f'Mean: {df[col].mean()}')
        print(f'Mode: {df[col].mode()}')
        print(f'Median: {df[col].median()}')
        print(f'{df[col].value_counts()}')
        print('-------')

In [14]:
# Using the function for sfh_train
v(sfh_train)

Column: bedroomcnt
Min: 0.0
Max: 25.0
Mean: 3.2877709142429565
Mode: 0    3.0
Name: bedroomcnt, dtype: float64
Median: 3.0
3.0     578721
4.0     380599
2.0     201382
5.0      90682
6.0      15260
1.0      13936
0.0       7876
7.0       2919
8.0        668
9.0        167
10.0        79
11.0        20
13.0        10
12.0         7
14.0         4
15.0         4
25.0         1
18.0         1
16.0         1
Name: bedroomcnt, dtype: int64
-------
Column: bathroomcnt
Min: 0.0
Max: 32.0
Mean: 2.2312968676127047
Mode: 0    2.0
Name: bathroomcnt, dtype: float64
Median: 2.0
2.00     565842
3.00     253894
1.00     248718
2.50      85953
4.00      49561
1.50      18814
3.50      17167
5.00      17066
4.50      11687
0.00       7800
6.00       6526
5.50       3716
7.00       2610
8.00       1030
6.50        803
9.00        423
7.50        236
10.00       189
11.00        79
8.50         59
12.00        46
9.50         27
13.00        26
14.00        13
15.00         9
16.00         8
0.50        

In [15]:
#random.randint(0, 0)

In [16]:
# Determining the range of values to input for bedroomcnt
# 2-5 bedrooms seems to encompass most of the data, so the remaining nulls will fall within those values
(578721+380599+201382+90682)/len(sfh_train['bedroomcnt'])

0.968307900159478

In [ ]:
sfh_train['bedroomcnt'].fillna(random.randint(2.0, 5.0), inplace = True)

In [ ]:
# Making sure it worked
sfh_train.isnull().sum()

In [ ]:
#sfh_train.bathroomcnt.value_counts()

In [ ]:
# This will output an integer value between 1.0 and 3.0, but can be 
random.randint(1.0, 3.0)

In [ ]:
sfh_train['bathroomcnt'].fillna(random.randint(1.0, 3.0), inplace = True)

In [ ]:
# I'm using 1624(median) as it falls between the mode and the mean values
sfh_train.calculatedfinishedsquarefeet.fillna(1624.0, inplace = True)

In [ ]:
#sfh_train.taxvaluedollarcnt.value_counts(bins=500)

In [ ]:
sfh_train['taxvaluedollarcnt'].fillna(450000.0, inplace = True)

In [ ]:
sfh_train.yearbuilt.value_counts(bins=10)

In [ ]:
sfh_train['yearbuilt'].fillna(random.randint(1908.0, 1994.0), inplace = True)

In [ ]:
sfh_train.taxamount.value_counts(bins=10)

In [ ]:
sfh_train['taxamount'].fillna(4108.0, inplace = True)

In [ ]:
sfh_train.head()

In [ ]:
# Checking one last time
sfh_train.isnull().sum()

In [ ]:
# Now to create the function that will be added to wrangle.py (only for 'bedroomcnt','bathroomcnt',
# 'calculatedfinishedsquarefeet','taxvaluedollarcnt','yearbuilt','taxamount','fips')

def wrangle_zillow(df):
    df.bedroomcnt.fillna(random.randint(2.0, 5.0), inplace = True)
    df.bathroomcnt.fillna(random.randint(1.0, 3.0), inplace = True)
    df.calculatedfinishedsquarefeet.fillna(df.calculatedfinishedsquarefeet.median(), inplace = True)
    df.taxvaluedollarcnt.fillna(df.taxvaluedollarcnt.mode(), inplace = True)
    # For yearbuilt I'll use 1958 as it falls in the middle of the mean and mode and they are all fairly close in value
    df.yearbuilt.fillna(df.yearbuilt.median(), inplace = True)
    df.taxamount.fillna(df.taxamount.median(), inplace = True)
    print(df)
    return df

In [ ]:
# testing the function on validate
wrangle_zillow(validate)

In [ ]:
validate.isnull().sum()

---

# More things that need to be done

In [ ]:
sfh_train.taxvaluedollarcnt.mode().max()

In [ ]:
train_sfh = pd.read_csv('sfh_zillow_train.csv', index_col=0)
train_sfh

In [ ]:
train_sfh = train_sfh[['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips']]
train_sfh.head()

In [ ]:
train_sfh.isnull().sum()

In [ ]:
train = wrangle.wrangle_7(train_sfh)

In [ ]:
train.isnull().sum()

In [ ]:
train.shape

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in train.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    train[col].hist()

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    plt.ticklabel_format(useOffset=False)
    
plt.show()

In [ ]:
plt.figure(figsize=(8,4))

plt.ticklabel_format(useOffset=False, style='plain')
sns.boxplot(data=train.drop(columns=['fips']))

plt.show()

In [ ]:
columns = ['bedroomcnt','bathroomcnt','calculatedfinishedsquarefeet','yearbuilt','taxamount']

plt.figure(figsize=(16, 20))

#plt.subplot(1, len(cols), plot_number)

sns.boxplot(data=train[columns])

plt.show()

In [ ]:
plt.figure(figsize=(16, 20))
# List of columns
#cols = [col for col in train.columns if col not in ['fips', 'yearbuilt']]
cols = [col for col in train.columns if col not in ['fips', 'yearbuilt']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=train[col])

    # Hide gridlines.
    plt.grid(False)

plt.show()

In [ ]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[col] > lower_bound) & (df[col] < upper_bound)]
        
    return df

In [ ]:
def remove_outliers_v2(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    # Create a column that will label our rows as containing an outlier value or not
    num_obs = df.shape[0]
    df['outlier'] = False
    for col in col_list:

        q1, q3 = df[col].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # update the outlier label any time that the value is outside of boundaries
        df['outlier'] = np.where(((df[col] < lower_bound) | (df[col] > upper_bound)) & (df.outlier == False), True, df.outlier)
    
    df = df[df.outlier == False]
    df.drop(columns=['outlier'], inplace=True)
    print(f"Number of observations removed: {num_obs - df.shape[0]}")
        
    return df

In [ ]:
zillow17.head(10)

In [ ]:
zillow = wrangle.wrangle_zillow(zillow17)

In [ ]:
zillow.head()

In [ ]:
zillow = remove_outliers(zillow, 1.5, ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'taxamount'])
zillow

In [ ]:
zillow.describe().T

In [ ]:
train_sfh, validate_sfh, test_sfh = wrangle.split(zillow)

### Going to create separate .csv for single family data

In [ ]:
train_sfh.to_csv('sfh_zillow_train.csv')
validate_sfh.to_csv('sfh_zillow_validate.csv')
test_sfh.to_csv('sfh_zillow_test.csv')

In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = [col for col in zillow.columns if col not in ['fips', 'year_built']]

for i, col in enumerate(cols):

    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display histogram for column.
    zillow[col].hist(bins=5)

    # Hide gridlines.
    plt.grid(False)
    
    # turn off scientific notation
    plt.ticklabel_format(useOffset=False)
    
    # mitigate overlap
    plt.tight_layout()
    
plt.show()

In [ ]:
# List of columns
cols = ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'taxamount']

plt.figure(figsize=(16, 3))

for i, col in enumerate(cols):

    # i starts at 0, but plot should start at 1
    plot_number = i + 1 

    # Create subplot.
    plt.subplot(1, len(cols), plot_number)

    # Title with column name.
    plt.title(col)

    # Display boxplot for column.
    sns.boxplot(data=zillow[[col]])

    # Hide gridlines.
    plt.grid(False)

    # sets proper spacing between plots
    plt.tight_layout()
    
plt.show()

### Testing lessons learned from review

In [ ]:
# Set our null threshold. Any columns that have this ratio or higher will be removed
missing_perc_thresh = 0.98

# Create empty list to keep track of which columns we plan on dropping
exclude_missing = []

# Find columns that have a greater null percentage than our threshold
num_rows = zillow.shape[0]
for c in zillow.columns:
    num_missing = zillow[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print("We exclude: %s" % exclude_missing)

# Drop these columns from our dataset
zillow.drop(columns=exclude_missing, inplace=True)

In [ ]:
imputer = SimpleImputer(strategy='median')  # build imputer

imputer.fit(train[['year_built']]) # fit to train

# transform the data
train[['year_built']] = imputer.transform(train[['year_built']])
validate[['year_built']] = imputer.transform(validate[['year_built']])
test[['year_built']] = imputer.transform(test[['year_built']])